# Data Processing

### Step 1: Load Libraries

In [ ]:
''' data_processing.py
This Jupyter notebook performs data transformation tasks, including 
- Converting all message JSON files into a single CSV file
- Cleaning and filtering data
- Extracting emojis from messages
- Adding datetime components
- Calculating character and emoji counts
- Performing sentiment analysis using VADER
- Generating emoji, unigram, bigram, and trigram counts
- Concudcting analysis with Word2Vec

The end result is a CSV file of all Instagram messages I have ever sent, enriched with sentiment and count data for analysis.

I created additional CSV files for 
- unigram counts
- bigram counts
- trigram counts
- emoji counts
- Word2Vec embeddings

This analysis is done almost entirely without AI for Steps 1-4, with the small exception of consulting ChatGPT5 for emoji unicoding advice and regex pattern syntax.
I consulted past NLTK and Word2Vec projects for technical implementation guidance for Step 3 and Step 5.
For Step 5, I consulted ChatGPT5 for Word2Vec implementation guidance and to debug my version of Word2Vec code. 
'''
import pandas as pd
import os
import json
import re
import nltk
from collections import Counter
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
import gensim
from gensim.models import Word2Vec
from typing import List, Optional, Tuple

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\yeahs\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


### Step 2: Parse Messages Data and Convert to CSV

In [42]:
inbox_path = 'inbox'
all_data = []

# Iterate through inbox directories and process message_1.json files
for root, dirs, files in os.walk(inbox_path):
    for file in files:
        if file == 'message_1.json':
            file_path = os.path.join(root, file)

            # Extract recipient name
            recipient_name = os.path.basename(root).split('_')[0]

            # Open and read JSON file
            with open(file_path, 'r', encoding='utf-8') as f:
                content = json.load(f)

                if isinstance(content, dict):
                    if 'messages' in content:
                        content = content['messages']
                    else:
                        content = [content]

                # Normalize JSON data into a DataFrame
                df = pd.json_normalize(content)
                df['recipient_name'] = recipient_name  
                all_data.append(df)

# Combine all DataFrames and clean data
merged_df = pd.concat(all_data, ignore_index=True)
merged_df = merged_df[merged_df['sender_name'] == 'Siddharth Chattoraj']
merged_df = merged_df[['recipient_name', 'timestamp_ms', 'content']]
merged_df.sort_values(by='timestamp_ms', inplace=True)
merged_df.to_csv('data/all_messages.csv', index=False)

### Step 3: Clean and Extract Data + Conduct Sentiment Analysis

In [43]:
all_messages = pd.read_csv('data/all_messages.csv')

# Clean emoji encoding issues
def fix_emoji(text):
    try:
        return text.encode('latin1').decode('utf-8')
    except Exception:
        return text
all_messages['content'] = all_messages['content'].apply(fix_emoji)
all_messages = all_messages.dropna(subset=['content']).reset_index(drop=True)
allowed_chars = r"a-zA-Z0-9\s.,!?;’:'\"()\[\]{}\-\–—_@#$/\\%&*+=<>~`|^…\u00C0-\u017F"
remove_symbols_re = re.compile(fr"[^{allowed_chars}]")
extract_symbols_re = re.compile(fr"[^{allowed_chars}]")

all_messages['emojis'] = all_messages['content'].apply(
    lambda x: extract_symbols_re.findall(x) if isinstance(x, str) else []
)
bad_emojis = {'️', '“', '”', '⃣', '\u200d'}
all_messages['emojis'] = all_messages['emojis'].apply(
    lambda x: [e for e in x if e not in bad_emojis]
)
all_messages['content'] = all_messages['content'].apply(
    lambda x: ''.join([c for c in x if c not in bad_emojis]) if isinstance(x, str) else x
)

# Fix emoji color issue
def restore_color(e):
    if not isinstance(e, str):
        return e
    e = e.replace("🇸", "").replace("🇺", "")
    if e in {"❤", "☹", "☺", "☘", "♥"} and not e.endswith("\ufe0f"):
        e = e + "\ufe0f"
    return e

all_messages['emojis'] = all_messages['emojis'].apply(
    lambda lst: [restore_color(e) for e in lst] if isinstance(lst, list) else lst
)
# drop empty/whitespace emoji tokens in the lists
all_messages['emojis'] = all_messages['emojis'].apply(
    lambda lst: [e for e in lst if isinstance(e, str) and e.strip()] if isinstance(lst, list) else lst
)

def restore_color_text(s):
    if not isinstance(s, str):
        return s
    return (
        s.replace("❤", "❤️")
         .replace("☹", "☹️")
         .replace("☺", "☺️")
         .replace("☘", "☘️")
         .replace("🇸", "")
         .replace("🇺", "")
    )

all_messages['content'] = all_messages['content'].apply(restore_color_text)

# Timestamps to datetime and extract components
dt_utc = pd.to_datetime(all_messages['timestamp_ms'], unit='ms', utc=True)
local_tz = 'America/Los_Angeles'
dt_local = dt_utc.dt.tz_convert(local_tz)
all_messages['datetime'] = dt_local
all_messages = all_messages.drop(columns=['timestamp_ms'])
all_messages = all_messages.sort_values(by='datetime').reset_index(drop=True)
all_messages['year'] = all_messages['datetime'].dt.year
all_messages['month'] = all_messages['datetime'].dt.month
all_messages['day'] = all_messages['datetime'].dt.day
all_messages['hour'] = all_messages['datetime'].dt.hour

# Add count of number of words and number of emojis
all_messages['word_count'] = all_messages['content'].apply(
    lambda x: len(x.split()) if isinstance(x, str) else 0
)
all_messages['character_count'] = all_messages['content'].apply(
    lambda x: len(x) if isinstance(x, str) else 0
)

all_messages['emoji_count'] = all_messages['emojis'].apply(len)

# Sentiment Analysis using VADER
sid = SentimentIntensityAnalyzer()
all_messages['compound_sentiment'] = all_messages['content'].apply(
    lambda x: sid.polarity_scores(x)['compound']
)

def label_sentiment(score):
    if score >= 0.05:
        return 'positive'
    elif score <= -0.05:
        return 'negative'
    else:
        return 'neutral'

all_messages['sentiment_label'] = all_messages['compound_sentiment'].apply(label_sentiment)
all_messages.to_csv('data/cleaned_messages.csv', index=False)

### Step 4: Unigrams, Bigrams, Trigrams, and Emojis

In [44]:
import ast

cleaned_messages = pd.read_csv('data/cleaned_messages.csv')

# Define cleaning
url_pattern = r"(https?://\S+|www\.\S+)"
punctuation_pattern = r"(^[^\w']+|[^\w']+$)"

def clean_for_ngrams(text):
    if not isinstance(text, str):
        return ''
    text = text.lower()
    text = re.sub(url_pattern, '', text)
    tokens = [re.sub(punctuation_pattern, '', t) for t in text.split()]
    return ' '.join(t for t in tokens if t)

cleaned_messages['content_further'] = cleaned_messages['content'].astype(str).apply(clean_for_ngrams)

# Add a date column (no timezone needed for daily grouping)
if 'date' not in cleaned_messages.columns:
    cleaned_messages['date'] = pd.to_datetime(
        cleaned_messages[['year', 'month', 'day']]
    ).dt.date

# Unigrams by date
tokens = cleaned_messages['content_further'].str.split()
uni_df = pd.DataFrame({'date': cleaned_messages['date'], 'unigram': tokens})
unigram_counts = (
    uni_df.explode('unigram')
          .dropna(subset=['unigram'])
          .groupby(['date', 'unigram'])
          .size()
          .reset_index(name='count')
          .sort_values(['date', 'count'], ascending=[True, False])
)
unigram_counts.to_csv('data/unigram_counts.csv', index=False)

# Bigrams by date
bigrams_per_row = tokens.apply(
    lambda w: [' '.join(p) for p in zip(w, w[1:])] if len(w) > 1 else []
)
bi_df = pd.DataFrame({'date': cleaned_messages['date'], 'bigram': bigrams_per_row})
bigram_counts = (
    bi_df.explode('bigram')
         .dropna(subset=['bigram'])
         .groupby(['date', 'bigram'])
         .size()
         .reset_index(name='count')
         .sort_values(['date', 'count'], ascending=[True, False])
)
bigram_counts.to_csv('data/bigram_counts.csv', index=False)

# Trigrams by date
trigrams_per_row = tokens.apply(
    lambda w: [' '.join(t) for t in zip(w, w[1:], w[2:])] if len(w) > 2 else []
)
tri_df = pd.DataFrame({'date': cleaned_messages['date'], 'trigram': trigrams_per_row})
trigram_counts = (
    tri_df.explode('trigram')
          .dropna(subset=['trigram'])
          .groupby(['date', 'trigram'])
          .size()
          .reset_index(name='count')
          .sort_values(['date', 'count'], ascending=[True, False])
)
trigram_counts.to_csv('data/trigram_counts.csv', index=False)

# Emoji counts by date
emojis_parsed = cleaned_messages['emojis'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else (x if isinstance(x, list) else [])
)
emoji_df = pd.DataFrame({'date': cleaned_messages['date'], 'emoji': emojis_parsed})
emoji_counts = (
    emoji_df.explode('emoji')
            .dropna(subset=['emoji'])
            .groupby(['date', 'emoji'])
            .size()
            .reset_index(name='count')
            .sort_values(['date', 'count'], ascending=[True, False])
)
emoji_counts.to_csv('data/emoji_counts.csv', index=False)

In [45]:
# Create Unigrams, Bigrams, Trigrams, and Emoji count CSVs for visualization by hour
tokens = cleaned_messages['content_further'].str.split()
uni_df = pd.DataFrame({'hour': cleaned_messages['hour'], 'unigram': tokens})
unigram_counts = (
    uni_df.explode('unigram')
          .dropna(subset=['unigram'])
          .groupby(['hour', 'unigram'])
          .size()
          .reset_index(name='count')
          .sort_values(['hour', 'count'], ascending=[True, False])
)
unigram_counts.to_csv('data/unigram_counts_hour.csv', index=False)

# Bigrams by hour
bigrams_per_row = tokens.apply(
    lambda w: [' '.join(p) for p in zip(w, w[1:])] if len(w) > 1 else []
)
bi_df = pd.DataFrame({'hour': cleaned_messages['hour'], 'bigram': bigrams_per_row})
bigram_counts = (
    bi_df.explode('bigram')
         .dropna(subset=['bigram'])
         .groupby(['hour', 'bigram'])
         .size()
         .reset_index(name='count')
         .sort_values(['hour', 'count'], ascending=[True, False])
)
bigram_counts.to_csv('data/bigram_counts_hour.csv', index=False)

# Trigrams by hour
trigrams_per_row = tokens.apply(
    lambda w: [' '.join(t) for t in zip(w, w[1:], w[2:])] if len(w) > 2 else []
)
tri_df = pd.DataFrame({'hour': cleaned_messages['hour'], 'trigram': trigrams_per_row})
trigram_counts = (
    tri_df.explode('trigram')
          .dropna(subset=['trigram'])
          .groupby(['hour', 'trigram'])
          .size()
          .reset_index(name='count')
          .sort_values(['hour', 'count'], ascending=[True, False])
)
trigram_counts.to_csv('data/trigram_counts_hour.csv', index=False)

# Emoji counts by hour
emojis_parsed = cleaned_messages['emojis'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else (x if isinstance(x, list) else [])
)
emoji_df = pd.DataFrame({'hour': cleaned_messages['hour'], 'emoji': emojis_parsed})
emoji_counts = (
    emoji_df.explode('emoji')
            .dropna(subset=['emoji'])
            .groupby(['hour', 'emoji'])
            .size()
            .reset_index(name='count')
            .sort_values(['hour', 'count'], ascending=[True, False])
)
emoji_counts.to_csv('data/emoji_counts_hour.csv', index=False)

### Step 5: Word2Vec

In [47]:
input_csv = "data/cleaned_messages.csv"
output_overall = "data/word2vec_3d.csv"
output_paths = {
    "date":  "data/word2vec_3d_by_date.csv",
    "month": "data/word2vec_3d_by_month.csv",
    "year":  "data/word2vec_3d_by_year.csv",
    "hour":  "data/word2vec_3d_by_hour.csv",
}

vector_size = 3
window_size = 5
num_workers = 4
use_skipgram = 1        # 1 = skip-gram, 0 = CBOW
random_seed = 42

min_sentences_per_group = 1
min_unique_tokens_per_group = 3

# 'none' | 'zscore' | 'minmax'
scale_mode = "zscore"

# used to stand in for NA/NaT when grouping
missing_marker = "∅"


# load & basic checks
df = pd.read_csv(input_csv)

if "content" not in df.columns:
    raise ValueError("Expected a 'content' column in cleaned_messages.csv")
df["content"] = df["content"].astype(str)


def ensure_time_columns(df_in: pd.DataFrame) -> pd.DataFrame:
    """Fill/derive date, month (1–12), year, hour from any reasonable timestamp columns."""
    df_local = df_in.copy()

    ts_candidates = ["timestamp", "created_at", "datetime", "date_time", "time"]
    ts_col = next((c for c in ts_candidates if c in df_local.columns), None)

    if ts_col is not None:
        ts = pd.to_datetime(df_local[ts_col], errors="coerce", utc=True).dt.tz_localize(None)
    else:
        if {"year", "month", "day"}.issubset(df_local.columns):
            ts = pd.to_datetime(
                dict(year=df_local["year"], month=df_local["month"], day=df_local["day"]),
                errors="coerce",
            )
        elif "date" in df_local.columns:
            ts = pd.to_datetime(df_local["date"], errors="coerce")
        else:
            raise ValueError(
                "No usable timestamp columns found. Provide one of "
                "[timestamp/created_at/datetime/date_time/time] or year+month+day or date."
            )

    df_local["date"] = ts.dt.date
    df_local["year"] = ts.dt.year.astype("Int64")
    df_local["month"] = ts.dt.month.astype("Int64")  # 1..12
    df_local["hour"] = ts.dt.hour.astype("Int64")
    return df_local


df = ensure_time_columns(df)


# tokenizer
url_pattern = re.compile(r"(https?://\S+|www\.\S+)")
punct_pattern = re.compile(r"(^[^\w']+|[^\w']+$)")

def tokenize(text: str) -> List[str]:
    """Lowercase, strip URLs/edge punctuation, drop numeric-only tokens."""
    text = text.lower()
    text = url_pattern.sub("", text)
    raw = text.split()
    cleaned = [punct_pattern.sub("", t) for t in raw]
    return [t for t in cleaned if t and not t.isnumeric()]


def build_sentences(subdf: pd.DataFrame) -> List[List[str]]:
    """Turn each message into a token list; ignore empty lines."""
    out: List[List[str]] = []
    for t in subdf["content"]:
        toks = tokenize(t)
        if toks:
            out.append(toks)
    return out


def scale_xyz(arr: np.ndarray, mode: str) -> np.ndarray:
    """Optionally normalize 3D coordinates for nicer downstream ranges."""
    if arr.size == 0:
        return arr
    if mode == "none":
        return arr
    if mode == "zscore":
        mu = arr.mean(axis=0, keepdims=True)
        sd = arr.std(axis=0, keepdims=True)
        sd[sd == 0] = 1.0
        return (arr - mu) / sd
    if mode == "minmax":
        mn = arr.min(axis=0, keepdims=True)
        mx = arr.max(axis=0, keepdims=True)
        span = (mx - mn)
        span[span == 0] = 1.0
        return (arr - mn) / span
    return arr


def train_group(sentences: List[List[str]]) -> Tuple[Optional[pd.DataFrame], Optional[Word2Vec]]:
    """Fit a tiny 3D token map for one group; return coordinates+counts and the fitted model."""
    if len(sentences) < min_sentences_per_group:
        return None, None

    vocab = Counter(tok for s in sentences for tok in s)
    if len(vocab) < min_unique_tokens_per_group:
        return None, None

    # keep vocab generous on small groups
    unique_terms = len(vocab)
    min_count = 1 if unique_terms < 50 else 2

    model = Word2Vec(
        sentences=sentences,
        vector_size=vector_size,
        window=window_size,
        min_count=min_count,
        workers=num_workers,
        sg=use_skipgram,
        seed=random_seed,
    )

    words = model.wv.index_to_key
    if not words:
        return None, model

    vecs = np.vstack([model.wv[w] for w in words])
    vecs = scale_xyz(vecs, scale_mode)

    # token frequencies (fall back to local vocab if needed)
    freqs: List[int] = []
    for w in words:
        try:
            freqs.append(int(model.wv.get_vecattr(w, "count")))
        except Exception:
            freqs.append(int(vocab.get(w, 0)))

    emb_df = (
        pd.DataFrame(
            {
                "word": words,
                "x": vecs[:, 0].astype(float),
                "y": vecs[:, 1].astype(float),
                "z": vecs[:, 2].astype(float),
                "frequency": freqs,
            }
        )
        .sort_values("frequency", ascending=False, kind="stable")
    )

    return emb_df, model


def is_nullish(x) -> bool:
    """True for NA/NaT/empty-like strings."""
    try:
        if pd.isna(x):
            return True
        if isinstance(x, str) and x.strip().upper() in ("NAT", "NA", "NONE", ""):
            return True
        return False
    except Exception:
        return False


def sort_group_keys(keys, group_col: str):
    """Sort group keys; send sentinel/null to the end. Dates sort by actual time."""
    def is_sentinel(k):
        return isinstance(k, str) and k == missing_marker

    if group_col == "date":
        def parse_date(k):
            if is_sentinel(k) or is_nullish(k):
                return (1, pd.Timestamp.max)
            try:
                return (0, pd.to_datetime(k))
            except Exception:
                return (0, str(k))
        return sorted(keys, key=parse_date)

    def parse_generic(k):
        if is_sentinel(k) or is_nullish(k):
            return (1, np.inf)
        return (0, k)

    try:
        return sorted(keys, key=parse_generic)
    except Exception:
        return sorted(
            keys,
            key=lambda k: (1, "") if (is_sentinel(k) or is_nullish(k)) else (0, str(k)),
        )


def safe_grouper(series: pd.Series) -> pd.Series:
    """Replace null-ish values with a visible sentinel to avoid categorical null issues."""
    s = series.astype(object, copy=True)
    mask = s.apply(is_nullish)
    s[mask] = missing_marker
    return s


def process_by(group_col: str, out_path: Optional[str] = None) -> pd.DataFrame:
    """Fit and export per-group maps for the given column; no hard caps."""
    tmp = df.copy()

    keys_series = safe_grouper(tmp[group_col])
    grouped = tmp.groupby(keys_series, sort=False, dropna=False)

    key_values = list(grouped.groups.keys())
    key_values = sort_group_keys(key_values, group_col)

    rows = []
    kept, skipped = 0, 0
    for key in key_values:
        sub = grouped.get_group(key)
        sents = build_sentences(sub)
        emb_df, _ = train_group(sents)
        if emb_df is None or emb_df.empty:
            skipped += 1
            print(f"[skip] {group_col}={key!r}  sentences={len(sents)}  rows={len(sub)}")
            continue
        emb_df.insert(0, group_col, key if key != missing_marker else None)
        rows.append(emb_df)
        kept += 1
        print(f"[keep] {group_col}={key!r}  vocab_rows={len(emb_df)}")

    cols = [group_col, "word", "x", "y", "z", "frequency"]
    if rows:
        out_df = pd.concat(rows, ignore_index=True)[cols]
        if out_path:
            out_df.to_csv(out_path, index=False)
        print(f"[done] {group_col}: kept {kept}, skipped {skipped}, wrote {len(out_df)} rows → {out_path}")
        return out_df
    else:
        empty_df = pd.DataFrame(columns=cols)
        if out_path:
            empty_df.to_csv(out_path, index=False)
        print(f"[done] {group_col}: nothing to write (kept {kept}, skipped {skipped}).")
        return empty_df


# overall map
print("=== Overall ===")
all_sentences = build_sentences(df)
overall_df, _ = train_group(all_sentences)
if overall_df is None:
    pd.DataFrame(columns=["word", "x", "y", "z", "frequency"]).to_csv(output_overall, index=False)
    print(f"[done] overall: corpus too small, wrote empty file → {output_overall}")
else:
    overall_df.to_csv(output_overall, index=False)
    print(f"[done] overall: wrote {len(overall_df)} rows → {output_overall}")

# grouped maps
print("=== By date ===")
_ = process_by("date", out_path=output_paths["date"])

print("=== By month ===")
_ = process_by("month", out_path=output_paths["month"])

print("=== By year ===")
_ = process_by("year", out_path=output_paths["year"])

print("=== By hour ===")
_ = process_by("hour", out_path=output_paths["hour"])

=== Overall ===
[done] overall: wrote 2967 rows → data/word2vec_3d.csv
=== By date ===
[skip] date=datetime.date(2017, 10, 23)  sentences=2  rows=2
[keep] date=datetime.date(2017, 10, 27)  vocab_rows=4
[skip] date=datetime.date(2017, 10, 29)  sentences=1  rows=1
[keep] date=datetime.date(2017, 11, 2)  vocab_rows=11
[keep] date=datetime.date(2017, 11, 4)  vocab_rows=5
[keep] date=datetime.date(2017, 11, 9)  vocab_rows=9
[skip] date=datetime.date(2017, 11, 11)  sentences=1  rows=1
[keep] date=datetime.date(2017, 11, 13)  vocab_rows=4
[keep] date=datetime.date(2017, 11, 16)  vocab_rows=8
[skip] date=datetime.date(2017, 11, 19)  sentences=1  rows=1
[keep] date=datetime.date(2017, 11, 20)  vocab_rows=14
[keep] date=datetime.date(2017, 11, 22)  vocab_rows=4
[keep] date=datetime.date(2017, 11, 23)  vocab_rows=15
[keep] date=datetime.date(2017, 11, 25)  vocab_rows=5
[keep] date=datetime.date(2017, 11, 27)  vocab_rows=3
[keep] date=datetime.date(2017, 11, 30)  vocab_rows=13
[keep] date=datetime